In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import register_matplotlib_converters

from scipy.stats import norm
import plotly.express as px

from factor_estimator import *
from asset_allocation import AssetAllocation
from diversification_analyzer import *
from index_data_handler import IndexDataHandler


from IPython.display import display

pd.options.mode.chained_assignment = None
register_matplotlib_converters()
%matplotlib inline

Factor investing is one of the most interesting approaches to "outperform the market" if the market is a usual market cap weighted portfolio (like MSCI World, MSCI ACWI, FTSE All World or to a degree a S&P500).
When diving into the literature one finds a whole host of factors which may be valuable. The most intersting ones come from Fama and French (Size (SmB), Value (HmL), Conservative (CvA) and Robustness (RmW). There are more out there like Momentum (Carhart, 1997) or Liquidity (Pástor, Stambaugh 2003).
Since the author of this analysis is based out of Germany, the amount of different investment options is limited. Thus we keep with the following selection, where the number represents the MSCI Index code:

In [2]:
start_date="19981130"
indexHandler = IndexDataHandler(start_date=start_date)
index_codes = {**indexHandler.get_available_indices("Developed"), **indexHandler.get_available_indices("US")}
index_codes.pop("Small-Cap")
index_codes.pop("High Dividend")
display(pd.DataFrame.from_dict(index_codes))


KeyError: 'High Dividend (Europe)'

The question asked in this analysis is: **How do I mix the ETFs for the indecies** above?

In general there are two approaches for factor investing. The integrated approach uses one etf where the stocks are picked to satisfy more then one factor at a time. Those ETFs are a one-stop solution, but have significantly higher costs. Also there is currently only a very limited selection available in Germany. The most important one is [iShares Edge MSCI World Multifactor](https://www.justetf.com/de-en/etf-profile.html?isin=IE00BZ0PKT83) which is based on the WORLD DIVERSIFIED MULTI-FACTOR index.
The TER for this is 0.5% and thus its ~0.35% higher than a market neutral MSCI World and ~0.25% higher than most 'single factor ETF'

The other approach is a *index of indicies* where you mix different ETFs to target more than a single factor. The reasoning behind that is, that factors are not stable over time and it is desireable to target more than one for continuity.

In this analysis we will try to estimate the 5 factors of the Fama French 5-Factor model. Those are MKT, SMB, HML, CWA and RMW. We then try to find the most suitable combination of ETFs for our purposes.

Let's get the data first. The data is directly taking from MSCI, and is thus available for a longer range. We examine data from 1998-2020, since this is the time span with daily data available. Every point is always normalized to the 01/01/1997, so we can look at it like we invested a euro back then.

In [ ]:
data = []
reload=False
#start_date="20080101"
for key in index_codes.keys():
    code = index_codes[key]["code"]
    print("reading ",key)
    d = indexHandler.get_historic_stock_data(code, reload=reload)
    d.rename(columns={"level_eod":key}, inplace=True)
    data.append(d)

df = data[0]
for i in range(1,len(data)):
    df = df.merge(data[i], how='left',left_index=True,right_index=True)

display(df)


To check the data and to understand, let's chart the data:

In [ ]:
df = df.reset_index()
plt.figure(figsize=(20, 12))
for k in index_codes.keys():

    plt.plot(df["date"],df[k], label=k)
plt.legend(prop={'size':24})
plt.show()

We can see that all factors outperform the MSCI World.
We can also see that the two best running factors are Momentum and Small-Cap (Value).

Lets now calculate the factors for each of our indicies. The idea is, that the change of market price w.r.t to the risk free return can be descriped as 
\begin{equation}
    r_f = a \cdot MKT+b \cdot SMB +c \cdot HML+d \cdot RMW+e \cdot CMA
\end{equation}
The original Fama French paper knew a,b,c,d and e for a given portfolio (those are metrics like P/E), and deterimed the influence factors.
We do it the other way around. We know the influence factors from the paper and want to determine the 'sizeness' of an index (so we determine a,b,c,d,e).

In [ ]:
factors = {}
fit_results = {}
for key in index_codes.keys():
    code = index_codes[key]["code"]
    print("calculating for ",code, key)
    fitted_factors, results, data = estimate(code,start_date=start_date,region=index_codes[key]["region"])

    factors[key] = fitted_factors
    fit_results[key] = results
    

Before we move forward, lets have a look at the quality of the fit:

In [ ]:
D={}
plt.figure(figsize=(12, 5))
for key in factors.keys():
    D[key] = fit_results[key].rsquared
plt.bar(*zip(*D.items()))
plt.xticks(rotation = 90)

plt.show()


From my perspective the r-squared value is relativily low.. The values of 0.4 are rather weak and maybe we should run further analysis here?
The original paper also quotes relativly low r² values. Maybe that's just normal?

We can also have a quick look at the global factorness of each index. Factorness is defined as
\begin{equation}
\phi = \sum_i f_i
\end{equation}
where $f_i$ is the individual factor. The key question of this analysis is if $\phi$ is the factor to optimize on or how to weight the individual contributions.

In [ ]:
D={}
plt.figure(figsize=(12, 5))
for key in factors.keys():
    D[key] = factors[key].sum()
plt.bar(*zip(*D.items()))
plt.xticks(rotation = 90)
plt.show()

What we can see here is kind of telling. The Multi-Factor index is not outstanding! If the Mult-Factor index is clearly surperior we would have expected a high value here.
Let's look into the details and the individual factor for each of the 5 factors of the model:

In [ ]:
fig, axs = plt.subplots(5)
i = 0

for factorname in ["MKT","SMB","HML","RMW","CMA"]:
    D = {}
    for key in factors.keys():
        D[key] = factors[key][factorname]
    axs[i].bar(*zip(*D.items()))
    axs[i].set_title(factorname)
    #axs[i].set_ylim(-0.2,0.5)
    axs[i].set_ylabel("factor coefficent")
    i+=1
fig.set_size_inches(18.5, 18)
fig.show()

And for completness, lets look at it the other way around and see it grouped by the index:

In [ ]:
fig, ax = plt.subplots(len(index_codes))

width = 0.35  # the width of the bars
i = 0
for key in factors.keys():
    D = {}
    for factorname in ["MKT","SMB","HML","RMW","CMA"]:
        D[factorname] = factors[key][factorname]
        
    ax[i].bar(*zip(*D.items()),label=key)
    ax[i].set_title(key)
    #ax[i].set_ylim(-0.2,0.5)
    ax[i].set_ylabel("factor coefficent")
    i+=1
    


fig.tight_layout()
fig.set_size_inches(10, 30)
fig.show()

What can we learn from here? First: Small Cap (Value) seams to be a beast! It offers exposure to the market, SMB and HML! And it does not even have negative contributions to RMW and CMA. Wow! One needs to remind oneselve, that Small Cap (Value) only has a [single ETF](https://www.justetf.com/de-en/etf-profile.html?groupField=index&from=search&isin=IE00BSPLC413) in Germany which is relativly expensive (0.3% TER)

# Factor Exposure and Expected Returns for Mixtures

Lets move into the optimization. We first define our factor matrix

In [ ]:
factor_matrix = pd.DataFrame.from_dict(factors).drop(["Intercept"],axis=0)

The factorness vector $\vec{f}$ can now be expressed as:
\begin{equation}
\vec{f} = \underline{F} \cdot \alpha
\end{equation}
Where $\underline{F}$ is the factor matrix and $\alpha$ our asset allocation vector.

**This boils down to the key open question: How to weight the different factors?** 

As an idea, lets take the average factor values from FF, scale them to a year and take them as a weightening factor. We can then just calculate the returns as:
\begin{equation}
    r = \vec{f} \cdot \vec{P}
\end{equation}
Where P holds the averages of the factors.

*Note: Maybe we want to take the monthly factor values here?*

*Note2: Just scaling it ignores the compound effect*

*Note3: The Small Cap Value is like a US ETF, so ideally one may want to use the higher US values here?*




In [ ]:

index_names = df.columns.tolist()
index_names.remove("date")
numberOfIndecies = len(index_names)
def factorness(alpha):
  return factor_matrix.dot(alpha)

average_factors = get_average_factors("Developed")*252
def excess_returns(alpha):
    factors = factorness(alpha)
    returns = average_factors.dot(factors)
    return returns*100
asset_allocations = {} # this will hold all proposals for later loops etc

Lets now define a few portfolios which can be interesting for us

In [ ]:
from asset_allocation import AssetAllocation

asset_allocations["Pure Multi"]= AssetAllocation(index_names).set_allocation_by_dict({"Multi-Factor":1})
asset_allocations["Pure World"]= AssetAllocation(index_names).set_allocation_by_dict({"MSCI World":1})
asset_allocations["Small-Value-Momentum"] = AssetAllocation(index_names).set_allocation_by_dict({"Small-Cap (Value)":0.4,"Momentum":0.2,"Value":0.4})
asset_allocations["Small-Momentum-Value"] = AssetAllocation(index_names).set_allocation_by_dict({"Small-Cap (Value)":0.4,"Momentum":0.4,"Value":0.2})
asset_allocations["Small-Quality-Value"] = AssetAllocation(index_names).set_allocation_by_dict({"Small-Cap (Value)":0.4,"Quality":0.2,"Value":0.4})
asset_allocations["Even Allocation"] = AssetAllocation(index_names).set_allocation_by_dict({"Value":0.2,"Quality":0.2,"Momentum":0.2,"Low Volatility":0.2,"Small-Cap (Value)":0.2})
asset_allocations["Momentum Tilt"] = AssetAllocation(index_names).set_allocation_by_dict({"MSCI World":0.6,"Momentum":0.4})
asset_allocations["Value Tilt"] = AssetAllocation(index_names).set_allocation_by_dict({"MSCI World":0.6,"Value":0.4})
asset_allocations["Even (No Vola)"] = AssetAllocation(index_names).set_allocation_by_dict({"Value":0.25,"Quality":0.25,"Momentum":0.25,"Small-Cap (Value)":0.25})

We created a few things. As the raw baseline we create the Pure World one, which is just the MSCI World. Then we have a pure Multi which is just the Multi factor one.

We define a few mixtures, an even allocation between the 5 factors as well as simple value/momentum tilted portfolios which have 60% MSCI world and 40% of the respective factor.

In [ ]:
result_dict = {}
#print(asset_allocations.keys())
for key in asset_allocations.keys():
    result_dict[key] = {}
    factorness_vector = factorness(asset_allocations[key].to_array())
    result_dict[key]["factorness"] = float(factorness_vector.sum())
    result_dict[key]["returns"] = float(excess_returns(asset_allocations[key].to_array())[0])
    for name, value in zip(["MKT","SMB","HML","RMW","CMA"],factorness_vector[0]):
        result_dict[key][name] = value

    
result_df = pd.DataFrame(result_dict).T.sort_values("returns",ascending=False)
display(result_df)

# Numerical Optimization for Returns
We can now try to use a numerical optimization in order to find the result which maximizes the total factorness $\phi$. One key thing here is, that we need to specifiy bonds. If not the optimizer will throw everything into Small Cap (Value), since it's a beast.
Lets define bounds as \[0, 0.4\] and lets also deactivate the use of multi factor:

In [ ]:

from scipy.optimize import minimize
# objective function
def func(alpha):
    return -1*excess_returns(alpha)

# constraint: sum(weights) = 1
fconst = lambda w: 1 - sum(w)
cons   = ({'type':'eq','fun':fconst})

# initial weights
w0 =  np.ones((len(index_codes),1))/len(index_codes)

# define bounds
b    = (0.0, 0.4) 
bnds = [b for i in range(0,numberOfIndecies)]
bnds[3] = (0.0,0.0001)
# minimize
sol_returns  = minimize(func,
                w0,
                bounds      = bnds,
                constraints = cons)
print(sol_returns.fun*-1)
return_optimize_dict = {}
for alloc, name in zip(sol_returns.x,factor_matrix.columns):
    return_optimize_dict[name]=alloc

display(pd.DataFrame.from_records(return_optimize_dict,index=["allocation"]).round(decimals=2))
asset_allocations["Return Optimized"] = AssetAllocation(index_names).set_allocations(sol_returns.x)



It is somewhat funny to see, that the result of this longer analys
is is a bit the trivial solution of taking the 3 best running ones and mixing them. The reason for this is that we do not yet consider volatility nor diversification as a constraint for our optimization.


In [ ]:
return_dict = {}
for key, value in asset_allocations.items():
    return_dict[key] = float(excess_returns(value.to_array()).iloc[0])
# normalize to MSCI World returns
pure_world_return = return_dict["Pure World"]
for key, value in return_dict.items():
    return_dict[key] = value-pure_world_return
    
display(pd.DataFrame.from_records(return_dict,index=["returns"]))
plt.bar(return_dict.keys(), return_dict.values())
plt.title("Excess Returns over MSCI World")
plt.xticks(rotation = 90)
plt.show()

In [ ]:
from factor_estimator import create_index_of_indices
for key, value in asset_allocations.items():
    df = create_index_of_indices(df,key,value)


In [ ]:
plt.figure(figsize=(20, 12))
allocs_to_show = [key for key in asset_allocations.keys()]
allocs_to_show.remove("Pure World")
allocs_to_show.append("Momentum")
plt.figure(figsize=(20, 12))

for k in allocs_to_show:
    plt.plot(df["date"],df[k], label=k)
#gfg = sns.lineplot(data = df[c])
plt.legend(prop={'size':24})
plt.show()


# Taking Volatility into Account

In [ ]:
def volatility(alpha):
    alloc = AssetAllocation(index_names)
    alloc.set_allocations(alpha)
    aux = create_index_of_indices(df,"aux",alloc)
    std_dev = aux["aux"].pct_change().dropna().std()*100
    return std_dev


volatility_dict = {}
for key in asset_allocations.keys():
    volatility_dict[key] = volatility(asset_allocations[key].to_array())#df[key].pct_change().dropna().std()*100
display(pd.DataFrame.from_records(volatility_dict,index=["daily standard deviation"]))


In [ ]:
from scipy.optimize import minimize
# objective function
regularizationFactor = 1
volatility_chart_dict = {}

def func(alpha):
    return -1*excess_returns(alpha).sum()+regularizationFactor*volatility(alpha)
results = {}
for regularizationFactor in np.arange(0,15,1):
    


    # constraint: sum(weights) = 1
    fconst = lambda w: 1 - sum(w)
    cons   = ({'type':'eq','fun':fconst})

    # initial weights
    w0 =  np.ones((len(index_codes),1))/len(index_codes)

    # define bounds
    b    = (0.0, .4) 
    bnds = [b for i in range(0,numberOfIndecies)]
    bnds[3] = (0.0,0.0001)
    # minimize
    sol  = minimize(func,
                    w0,
                    bounds      = bnds,
                    constraints = cons)

    optimized_allocation = AssetAllocation(index_names)
    optimized_allocation = optimized_allocation.set_allocations(sol.x)
#     print("### "+str(regularizationFactor)+" ###")
#     print(excess_returns(sol.x).sum(),"  /  ",volatility(sol.x))
#     print(optimized_allocation)
    results[regularizationFactor] = optimized_allocation.allocations
    results[regularizationFactor]["Returns"] = excess_returns(sol.x).sum()
    results[regularizationFactor]["Volatility"] = volatility(sol.x)

fit_results = pd.DataFrame.from_dict(results)
for c in fit_results.columns:
    fit_results[c] = fit_results[c].round(decimals=2)
display(fit_results)

# Add the volatility optimized allocation to our main data frame
volatility_allocation = AssetAllocation(index_names)
for name, alloc in results[10].items():
    if(name != "Returns" and name != "Volatility"):
        volatility_allocation.set_allocation(name, alloc)

asset_allocations["Volatility Optimized"] = volatility_allocation
df = create_index_of_indices(df,"Volatility Optimized",volatility_allocation)

comparison_dict = {"Pure World":{},"Pure Multi":{}}
comparison_dict["Pure World"]["Returns"] = float(excess_returns(asset_allocations["Pure World"].to_array()).sum())
comparison_dict["Pure World"]["Volatility"] = float(volatility(asset_allocations["Pure World"].to_array()))
comparison_dict["Pure Multi"]["Returns"] = float(excess_returns(asset_allocations["Pure Multi"].to_array()).sum())
comparison_dict["Pure Multi"]["Volatility"] = float(volatility(asset_allocations["Pure Multi"].to_array()))
comparison_df = pd.DataFrame.from_dict(comparison_dict)
display(comparison_df)

In [ ]:
plot_df = fit_results.T[["Returns","Volatility"]]
plt.figure(figsize=(15, 4))
plt.title("Volatility Analysis",fontsize=30)
plt.xlabel("Volatility",fontsize=24)
plt.ylabel("Returns",fontsize=24)
plt.grid(True)
markersize=12
plt.plot(plot_df["Volatility"], plot_df["Returns"],'bo',label="optimization",markersize=markersize)

for key, alloc in asset_allocations.items():
    if(key != "Small-Value-Momentum"):
        plt.plot(volatility(alloc.to_array()), excess_returns(alloc.to_array()).sum(),"o",label=key,markersize=markersize)
plt.legend(loc="lower right")
plt.show()

What can we learn from here? Well, we have a few interesting portfolios i think:

## The High Risk High Return Portfolio:
Thats the top right. It is
* 40% Small Cap (Value)
* 40% Value
* 20% Momentum
This gives you roughly 7.35% returns or 1.6% gain over the MSCI World. You pay for it with higher Volatility. 

## The Low Volatility
Thats on the lower left (there are a few, thats the "10")::
* 33% Low Volatility
* 10% Momentum
* 32% Quality
* 18% Value

You can get a roughly 6.5% return (remember, this is compared to US Treasury bonds). Which is 0.7% gain over the MSCI World or roughly the same as the Multi-Factor. 

# Looking into Volatility

For now, we will focus on the 'High Rish High Return Portfolio". How well did it run for the past years? Those cummulated plots above are nice to look at, because you mostly on top of the MSCI World curve, but it hides a bit the variability of the factor. Lets there fore look at the comparison to the MSCI World. When would we be worse or better off than the MSCI World?
To do this we have a look at:
\begin{equation}
 r = MA(\%\text{Change}(\text{daily_returns}))_{x} -MA(\%\text{Change}(\text{daily_returns}))_{MSCI World}
\end{equation}
Where MA is the moving average and %Change is the percent change to the day before.

Let's first have a look at the charts for the 'normal Factors' and then for our allocations

In [ ]:
plt.figure(figsize=(20, 12))
allocs_to_show = []
allocs_to_show.append("Momentum")
allocs_to_show.append("MSCI World")
allocs_to_show.append("Value")
allocs_to_show.append("Quality")
plt.figure(figsize=(20, 6))
window_size = 60
for k in allocs_to_show:
    plt.plot(df['date'],df[k].pct_change().rolling(window_size).mean()-df["MSCI World"].pct_change().rolling(window_size).mean(),label=k)
plt.legend(prop={'size':12})
plt.xlabel('date', fontsize=18)
plt.ylabel('Moving avg of % change - Moving avg of % change of MSCI World', fontsize=18)
plt.title("Comparison of selected allocations to MSCI World",fontsize=32)
plt.show()


In [ ]:
plt.figure(figsize=(20, 12))
allocs_to_show = [key for key in asset_allocations.keys()]

allocs_to_show.remove("Even Allocation")
allocs_to_show.remove("Small-Quality-Value")
allocs_to_show.remove("Return Optimized")
allocs_to_show.remove("Small-Momentum-Value")
#allocs_to_show = ["MSCI World","Momentum Tilt"]
plt.figure(figsize=(20, 6))
window_size = 90
for k in allocs_to_show:
    plt.plot(df['date'],df[k].pct_change().rolling(window_size).mean()-df["MSCI World"].pct_change().rolling(window_size).mean(),label=k)
plt.legend(prop={'size':12})
plt.xlabel('date', fontsize=18)
plt.ylabel('Moving avg of % change - Moving avg of % change of MSCI World', fontsize=18)
plt.title("Comparison of selected allocations to MSCI World",fontsize=32)
plt.show()

What we can learn from here, is that there will be always times where a given factor under performs the market. This is totally normal and it *has* to happen. Why? For more returns we get more volatility. More volatility means we are sometimes below the MSCI World, which is less volatile. Thats the point!

Lets look at this volatility more on a pratical level. The first two weeks after I bought my first ETF I was checking my stock app hourly. What a blockbuster!
This was a while ago and now I act more - passive. But still, most people aren't blind for the short term volatility of their stocks. Before we check the volatility defined as the daily standard deviation.
Now let's check the monthly standard deviation in comparison to the MSCI World. How often can we beat it?

In [ ]:
monthly_index_handler = IndexDataHandler(start_date=start_date,frequency="END_OF_MONTH",normalize=True)

monthly_data = []

#start_date="20080101"
for key in index_codes.keys():
    code = index_codes[key]["code"]
    print("reading", key)
    d = monthly_index_handler.get_historic_stock_data(code)
    d.rename(columns={"level_eod":key}, inplace=True)
    d = d.dropna()
    monthly_data.append(d)

monthly_df = monthly_data[0]
for i in range(1,len(monthly_data)):
    monthly_df = monthly_df.merge(monthly_data[i], how='left',left_index=True,right_index=True)
monthly_df = monthly_df.drop(monthly_df.filter(regex='_y$').columns.tolist(),axis=1)


for key, value in asset_allocations.items():
    monthly_df = create_index_of_indices(monthly_df,key,value)

            

In [ ]:
histo_dict = {}
for key in asset_allocations.keys():
    normalized_change_data = monthly_df[key].pct_change()-monthly_df["MSCI World"].pct_change()
    normalized_change_data = normalized_change_data.dropna()
    normalized_change_data = normalized_change_data*100
    mu = normalized_change_data.mean()

    histo_dict[key] ={}
    histo_dict[key]["mu"] = normalized_change_data.mean()
    histo_dict[key]["sigma"] = normalized_change_data.std()
    histo_dict[key]["kurtosis"] = normalized_change_data.kurtosis()
    histo_dict[key]["Values>0 [%]"] = normalized_change_data[normalized_change_data>0].size/normalized_change_data.size*100
    if(key=="Small-Value-Momentum"):
        plt.title("Small-Value-Momentum")
        plt.xlabel("Percent Gain over MSCI World per Month")
        plt.hist(normalized_change_data,bins=30)
plt.show()
hist_df = pd.DataFrame.from_dict(histo_dict)
display(hist_df)


    

The mean of the distribution is not too surprising. Our favourite of "Small-Value-Momentum" is in average 0.33\% a month above MSCI World. But this is only _in average_ . In 45% of the cases the MSCI World runs better in a given month. This is partly caused by the kurtosis, which is != 0 for all our ideas.

So this means? If you check a month and compare yourself with the MSCI World it tells you nothing. It's literally a coin flip. You need to get sample size to see that its works - which is a somewhat not surprising result - but good that we checked.


## Value and Volatility Lessons


If you look at the Value premium in our first chart you can see that it had a hard time in the time after 2016. Its very well known that Value investing was problematic over the past years. Is that a reason to abondon value?

Fama and French show in their 2018 paper [Volatility Lessons](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3081101), that it is totally normal that a factor under performs for 5 years. Their analysis, where there bootstrap themselves a lot of pseudo-dataset, they showed that a 5 year underperformance of a factor is nothing rare. Even a 10 year underperformance is not impossible.

Another great paper on the topic is [Reports of Value’s Death May Be Greatly Exaggerated](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3488748). The authors come to the same conclusion as FF above, saying that the recent 'slump' is a strong statistical abnormally, but in 2.5% of their bootstrapped samples they expirienced a similar or worse situation.
The authors also show, that most of the arguments by 'Death-Callers' are easily refutable. They however say, that P/E is a bad ratio to start with and that there are some problems with high investments into R&D, which are likely not covered by earnings.

We will always see 'finance pornographs' call a factor dead. But as factor investors we need to make ourselves aware, that (to the best of my knowledge), there is no statistical evidence for this. This is in my opinion the pain you need to go throw, to get the better returns.


# The Momentum Value Trade Off

Our analysis showed, that it may be very intersting to invest into momentum and value at the same time. After doing the analysis I saw the paper [Value and Momentum everywhere](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1363476), which supports this idea. I would definitly advise you to read it, here are two short quotes from the conclusion:

*Value and momentum deliver positive expected abnormal returns in a variety of markets and  asset  classes,  their  combination  performs  even  better  than  either  alone,  and  the  benefits of diversification across markets and asset classes are large*

and

*We find that value (and momentum) strategies are positively related across markets and asset  classes  and  that  value  and  momentum  are  negatively  related  within  and  across  markets  and  asset  classes*

The paper also shows a negative correlation between momentum and value (see Table 1), which is desirable. If two factors are negativly correlated with one another, then they cover for the other when the other is week. Thus you get a more steady total return curve and hopefully it is then easier to stay invested.

# 20% Momentum? Or more?

So, we setteled down to a portfolio with 40% Small Cap (Value) and the rest is Momentum and Value. The optimization yielded 40% Value and 20% Momentum.
The question is: Is that correct?

Lets therefore look at two charts.
* 1. The raw delta between the 40% Value and other allocations
* 2. The same metric we used enough, the moving average in change of daily gains in percent.


In [ ]:
momentum_df = df[["date","Small-Value-Momentum"]]

m_alloc = AssetAllocation(index_names)
m_alloc.set_allocation("Small-Cap (Value)",0.4)

fig, ax = plt.subplots(2)

fig.tight_layout()
fig.set_size_inches(20, 15)
window_size = 90
for value_alloc in [0.1,0.2,0.3,0.4,0.5]:
    m_alloc.set_allocation("Value",value_alloc)
    m_alloc.set_allocation("Momentum",0.6-value_alloc)
    name = "Value: "+str(value_alloc)

    momentum_df.loc[:,name] = create_index_of_indices(df,name,m_alloc).loc[:,name]
    momentum_df[name+"_relative"] =  momentum_df[name].pct_change().rolling(window_size).mean() - df["Small-Value-Momentum"].pct_change().rolling(window_size).mean()

    ax[0].plot(momentum_df['date'],momentum_df[name]-df["Small-Value-Momentum"],label=name)
    ax[1].plot(momentum_df['date'],momentum_df[name+"_relative"],label=name)
ax[0].set_title("Different Value Ratios compared to 40% Value")
ax[1].set_title("Yearly Moving Average of percent change compared to 40% Value")
plt.legend(prop={'size':24})#
plt.show()


What can we learn from here? For a long long time it would hav eeen beneficial to go in with as much value as possible. Only in 2018 forward Momentum 'kicked back' and is now, even in total, on top.
The problem I see here is the sample bias problem. In the end we only work on 22 years of data. If - by accident - the first 15 of them where strong value years our strategy is directled towards more value then it would be good over all. 

# Holding Analysis

In [ ]:
holding_indices = index_codes
data = []
index_names = [ key for key in holding_indices.keys()]
spdr = None
for key in holding_indices:
    if(holding_indices[key]["vendor"] == "Xtrackers"):
        data.append(get_xtrackers(holding_indices[key]["ISIN"],key))
    if(holding_indices[key]["vendor"] == "SPDR"):
        data.append(get_spdr(holding_indices[key]["ticker"],key))
    if (holding_indices[key]["vendor"] == "iShares"):
        #print("Will not read ",key,"iShares is annoying")
        #does not work yet, isharse names INTEL as INTEL Corp etc -.-
        data.append(get_iShares(key,key))
        

holding_df = data[0]
for i in range(1,len(data)):
    #print(data[i])
    holding_df = holding_df.merge(data[i], how='outer',left_on="ISIN",right_on="ISIN",suffixes=('', '_DROP'))
    holding_df["Name"].fillna(holding_df["Name_DROP"], inplace=True)
    holding_df["Country"].fillna(holding_df["Country_DROP"], inplace=True)
    holding_df["Industry Classification"].fillna(holding_df["Industry Classification_DROP"], inplace=True)
    holding_df = holding_df.fillna(0).filter(regex='^(?!.*_DROP)')

## Top Holdings

In [ ]:
chosen_allocation = AssetAllocation(index_names)

chosen_allocation.set_allocation("Momentum",0.2)
chosen_allocation.set_allocation("Value",0.4)
chosen_allocation.set_allocation("Small-Cap (Value)",0.4)
holding_df = create_index_of_indices(holding_df, "Small-Value-Momentum", chosen_allocation)


holding_df = holding_df.sort_values("Small-Value-Momentum",ascending=False)
#display(holding_df[["Name","Small-Value-Momentum"]].head(50))

fig = px.bar(holding_df.head(30), x='Name', y='Small-Value-Momentum')
fig.show()

In [ ]:
grouped = holding_df.groupby("Industry Classification").sum()
grouped = grouped.reset_index()
grouped["Industry Classification"][grouped["Small-Value-Momentum"] <0.01] = "Other"
#print(grouped)
fig = px.pie(grouped, values='Small-Value-Momentum', names='Industry Classification')
fig.show()

In [ ]:
grouped = holding_df.groupby("Country").sum()
grouped = grouped.reset_index()
grouped["Country"].loc[grouped["Small-Value-Momentum"] <0.01] = "Other"
#print(grouped)
fig = px.pie(grouped, values='Small-Value-Momentum', names='Country')
fig.show()

# The European Question


From the holding analysis above we learned, that  the current portfolio is tilted towards the US by 70%. As a comparison - the normal MSCI World has a ~66% US stocks.
One may ask the question if it makes sense to enhance the european part of the portfolio to get more independend to the US. We can quickly examine historical performance of the corresponding European indices for Momentum, Value and Small Cap Value:

european_indices = indexHandler.get_available_indices(region="Europe")
em_indices = indexHandler.get_available_indices(region="EM")

In [ ]:
european_indices = indexHandler.get_available_indices("Europe")
def compare_global_to_local(factor="Value"):
    euro_df = indexHandler.get_historic_stock_data(european_indices[factor+" (Europe)"]["code"])
    euro_df = euro_df.rename(columns={"level_eod":factor+" (Europe)"})
   
    comparison_df = euro_df.merge(df[[factor,"MSCI World","date"]],how='inner',left_on="date",right_on="date")
    
    plt.figure(figsize=(20, 6))
    for k in [factor,factor+" (Europe)","MSCI World"]:
        plt.plot(comparison_df['date'],comparison_df[k],label=k)
    plt.legend(prop={'size':12})
    plt.xlabel('date', fontsize=18)
    plt.ylabel('', fontsize=18)
    plt.show()


In [ ]:
compare_global_to_local("Momentum")

In [ ]:
compare_global_to_local("Value")

In [ ]:
compare_global_to_local("Small-Cap (Value)")

What we see is that the european index is in all 3 cases worse than the Global one (or in the Small-Cap one, the european one).
The value one basically gets outperformed like crazy since the financial crises in 2008. But then again, thats when Value got hit hard.
Momentum went through the roof since 2015ish in the world variant but not in europe. Instead there was barely any gain!

The Small-Cap (Value) one may be the most intersting one to diversify. Here the EU one even outperformed the US one pre financial crises.

## Emerging Markets

In [ ]:
em_indices = indexHandler.get_available_indices("EM")
def compare_em_to_local(factor="Value"):
    em_df = indexHandler.get_historic_stock_data(em_indices[factor+" (EM)"]["code"])
    em_df = em_df.rename(columns={"level_eod":factor+" (EM)"})
   
    comparison_df = em_df.merge(df[[factor,"MSCI World","date"]],how='inner',left_on="date",right_on="date")
    
    plt.figure(figsize=(20, 6))
    for k in [factor,factor+" (EM)","MSCI World"]:
        plt.plot(comparison_df['date'],comparison_df[k],label=k)
    plt.legend(prop={'size':12})
    plt.xlabel('date', fontsize=18)
    plt.ylabel('', fontsize=18)
    plt.show()


In [ ]:
compare_em_to_local("Value")

In [ ]:
compare_em_to_local("Momentum")

In [ ]:
compare_em_to_local("Small-Cap (Value)")

# Known Drawbacks - Limitations and FAQ

## The data is only back to 1998

## Isn't Value dead?

## Value has a lot of crap in it

## Small Cap Value's highest holding is GME - isn't that crazy?


## There is only this single SPDR Small Cap Value ETF - isn't it good?

## What about High Dividends?

